In [1]:
!pip install SimpleITK
!pip install diskcache
from collections import namedtuple
import functools
import glob
import os
import csv
import numpy as np
import SimpleITK as sitk
import gzip
from diskcache import FanoutCache, Disk
# from diskcache.core import BytesType, MODE_BINARY, BytesIO

import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim
import torch.functional as F



     |████████████████████████████████| 42.5MB 93kB/s 
     |████████████████████████████████| 51kB 1.8MB/s 


# DATA DOWNLOAD

In [2]:
!cp -r /content/drive/My\ Drive/PyTorch/pytorch_start/luna_download_extract.sh /content


In [3]:
!bash luna_download_extract.sh


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libevent-2.1-6 libutempter0
The following NEW packages will be installed:
  libevent-2.1-6 libutempter0 tmux
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 390 kB of archives.
After this operation, 1,068 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libevent-2.1-6 amd64 2.1.8-stable-4build1 [133 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libutempter0 amd64 1.1.6-3 [7,898 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 tmux amd64 2.6-3ubuntu0.2 [248 kB]
Fetched 390 kB in 1s (533 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed,

In [4]:
!tmux ls


subset0: 1 windows (created Mon Sep 14 03:09:33 2020) [80x24]
subset1: 1 windows (created Mon Sep 14 03:09:33 2020) [80x24]


In [17]:
!du -kh subset1/subset1.zip


5.9G	subset1/subset1.zip


In [8]:
!wc -l candidates.csv
!head candidates.csv


551066 candidates.csv
seriesuid,coordX,coordY,coordZ,class
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-56.08,-67.85,-311.92,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,53.21,-244.41,-245.17,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,103.66,-121.8,-286.62,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-33.66,-72.75,-308.41,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-32.25,-85.36,-362.51,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-26.65,-203.07,-165.07,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-74.99,-114.79,-311.92,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-16.14,-248.61,-239.55,0
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,135.89,-141.41,-252.2,0


In [10]:
!wc -l annotations.csv
!head annotations.csv


1187 annotations.csv
seriesuid,coordX,coordY,coordZ,diameter_mm
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,-128.6994211,-175.3192718,-298.3875064,5.651470635
1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860,103.7836509,-211.9251487,-227.12125,4.224708481
1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793540579077826395208,69.63901724,-140.9445859,876.3744957,5.786347814
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,-24.0138242,192.1024053,-391.0812764,8.143261683
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,2.441546798,172.4648812,-405.4937318,18.54514997
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,90.93171321,149.0272657,-426.5447146,18.20857028
1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405,89.54076865,196.4051593,-515.0733216,16.38127631
1.3.6.1.4.1.14519.5.2.1.6279.6001.100953483028192176989979435275,81.50964574,54.9572186,-150.3464233,10.36232088
1.3.6.1.4.1.14519.5.2.1.

# DATA CLEANING AND SANITY

In [5]:
Candidate_info_tuple = namedtuple("CandidateInfoTuple", "isNodule_bool, diameter_mm,\
                                   series_uid, center_xyz")

@functools.lru_cache(1)
def get_candidate_info_list(require_on_disk=True):
  mhd_list = glob.glob("subset*/*.mhd")
  present_on_disk = {os.path.split(p)[-1][:-4] for p in mhd_list}

  diameter_dict = {}
  with open("annotations.csv", "r") as f:
    for row in list(csv.reader(f))[1:]:
      series_uid = row[0]

      annotations_center_xyz = tuple(map(float, row[1:4]))
      annotations_diameter_mm = float(row[4])

      diameter_dict.setdefault(series_uid, []).append((annotations_center_xyz,
                                                       annotations_diameter_mm))
      
  candidates_info_list = []
  with open("candidates.csv", "r") as f:
    for row in list(csv.reader(f))[1:]:
      series_uid = row[0]

      if series_uid not in present_on_disk and require_on_disk:
        continue
      
      is_nodule_bool = bool(int(row[4]))
      candidate_center_xyz = tuple(map(float, row[1:4]))

      candidate_diameter_mm = 0.0
      for annotation_tuple in diameter_dict.get(series_uid, []):
        annotations_center_xyz, annotations_diameter_mm = annotation_tuple

        for i in range(3):
          delta_mm = abs(annotations_center_xyz[i] - candidate_center_xyz[i])

          if delta_mm > (annotations_diameter_mm/4):
            break
          
        else: # Only executes if "for" loop does not uses "break", i.e it runs smoothly without having to use break
          candidate_diameter_mm = annotations_diameter_mm
      
      candidates_info_list.append(
          Candidate_info_tuple(is_nodule_bool, candidate_diameter_mm,
                               series_uid, candidate_center_xyz))

  candidates_info_list.sort(reverse=True)
  return candidates_info_list


## UTILS.PY

In [6]:
Irc_tuple = namedtuple("IrcTuple", ["index", "row", "col"])
Xyz_tuple = namedtuple("XyzTuple", ["x", "y", "z"])

def irc2xyz(coord_irc, origin_xyz, vx_size_xyz, direction_a):

  cri_a = np.array(coord_irc[::-1])
  origin_a = np.array(origin_xyz)
  vx_size_xyz_a = np.array(vx_size_xyz)
  coords_xyz = (direction_a @ (cri_a * vx_size_xyz_a)) + origin_a
  return Xyz_tuple(*coords_xyz)

def xyz2irc(coord_xyz, origin_xyz, vx_size_xyz, direction_a):
  origin_a = np.array(origin_xyz)
  vx_size_a = np.array(vx_size_xyz)
  coord_a = np.array(coord_xyz)
  cri_a = ((coord_a - origin_a) * np.linalg.inv(direction_a)) / vx_size_a
  return Irc_tuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))


### DISK UTILS

In [7]:
class GzipDisk(Disk):
    def store(self, value, read, key=None):
        """
        Override from base class diskcache.Disk.

        Chunking is due to needing to work on pythons < 2.7.13:
        - Issue #27130: In the "zlib" module, fix handling of large buffers
          (typically 2 or 4 GiB).  Previously, inputs were limited to 2 GiB, and
          compression and decompression operations did not properly handle results of
          2 or 4 GiB.

        :param value: value to convert
        :param bool read: True when value is file-like object
        :return: (size, mode, filename, value) tuple for Cache table
        """
        # pylint: disable=unidiomatic-typecheck
        if type(value) is BytesType:
            if read:
                value = value.read()
                read = False

            str_io = BytesIO()
            gz_file = gzip.GzipFile(mode='wb', compresslevel=1, fileobj=str_io)

            for offset in range(0, len(value), 2**30):
                gz_file.write(value[offset:offset+2**30])
            gz_file.close()

            value = str_io.getvalue()

        return super(GzipDisk, self).store(value, read)


    def fetch(self, mode, filename, value, read):
        """
        Override from base class diskcache.Disk.

        Chunking is due to needing to work on pythons < 2.7.13:
        - Issue #27130: In the "zlib" module, fix handling of large buffers
          (typically 2 or 4 GiB).  Previously, inputs were limited to 2 GiB, and
          compression and decompression operations did not properly handle results of
          2 or 4 GiB.

        :param int mode: value mode raw, binary, text, or pickle
        :param str filename: filename of corresponding value
        :param value: database value
        :param bool read: when True, return an open file handle
        :return: corresponding Python value
        """
        value = super(GzipDisk, self).fetch(mode, filename, value, read)

        if mode == MODE_BINARY:
            str_io = BytesIO(value)
            gz_file = gzip.GzipFile(mode='rb', fileobj=str_io)
            read_csio = BytesIO()

            while True:
                uncompressed_data = gz_file.read(2**30)
                if uncompressed_data:
                    read_csio.write(uncompressed_data)
                else:
                    break

            value = read_csio.getvalue()

        return value

def getCache(scope_str):
    return FanoutCache('data-unversioned/cache/' + scope_str,
                       disk=GzipDisk,
                       shards=64,
                       timeout=1,
                       size_limit=3e11,
                       # disk_min_file_size=2**20,
    )


## UTILS END

In [8]:
class CT:
  def __init__(self, series_uid):
    mhd_path = glob.glob("subset*/{}.mhd".format(series_uid))[0]
    ct_mhd = sitk.ReadImage(mhd_path)
    ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

    ct_a.clip(-1000,1000,ct_a)

    self.series_uid = series_uid
    self.hu_a = ct_a

    self.origin_xyz = Xyz_tuple(*ct_mhd.GetOrigin())
    self.vx_size_xyz = Xyz_tuple(*ct_mhd.GetSpacing())
    self.direction_a = np.array(ct_mhd.GetDirection()).reshape(3,3)
  
  def get_raw_candidate(self, center_xyz, width_irc):
    center_irc = xyz2irc(center_xyz, self.origin_xyz, self.vx_size_xyz,
                         self.direction_a)
    
    slice_list = []
    for axis, center_val in enumerate(center_irc):
      start_ndx = int(round(center_val - width_irc[axis]/2))
      end_ndx = int(start_ndx + width_irc[axis])

      assert center_val >= 0 and center_val < self.hu_a.shape[axis], repr([self.series_uid, center_xyz, self.origin_xyz, self.vxSize_xyz, center_irc, axis])

      if start_ndx < 0:
        start_ndx = 0
        end_ndx = int(width_irc[axis])
      
      if end_ndx > self.hu_a.shape[axis]:
        end_ndx = self.hu_a.shape[axis]
        start_ndx = int(self.hu_a.shape[axis] - width_irc[axis])

      slice_list.append(slice(start_ndx, end_ndx))
    
    ct_chunk = self.hu_a[tuple(slice_list)]
    return ct_chunk, center_irc


class LunaDataset(Dataset):

  def __init__(self, val_stride=0, is_val_set_bool=None, series_uid=None):
    self.candidate_info_list = copy.copy(get_candidate_info_list())

    if series_uid:
      self.candidate_info_list = [x for x in self.candidate_info_list if \
                                  x.series_uid == series_uid]
    
    if is_val_set_bool:
      assert val_stride > 0, val_stride
      self.candidate_info_list = self.candidate_info_list[::val_stride]
      assert self.candidate_info_list
    
    elif val_stride > 0:
      del self.candidate_info_list[::val_stride]
      assert self.candidate_info_list

  def __len__(self):
    return len(self.candidate_info_list)
  
  def __getitem__(self, ndx):
    candidate_info_tup = self.candidate_info_list[ndx]
    width_irc = (32, 48, 48)

    candidate_a, center_irc = get_ct_raw_candidate(
        candidate_info_tup.series_uid, 
        candidate_info_tup.center_xyz,
        width_irc)
    
    candidate_t = torch.from_numpy(candidate_a)
    candidate_t = candidate_t.to(torch.float32)
    candidate_t = candidate_t.unsqueeze(0)

    pos_t = torch.tensor([not candidate_info_tup.isNodule_bool,
                          candidate_info_tup.isNodule_bool],
                         dtype=torch.long)
    
    return (candidate_t, pos_t, candidate_info_tup.series_uid, 
            torch.tensor(center_irc))
    
@functools.lru_cache(1, typed=True)
def get_ct(series_uid):
  return Ct(series_uid)

raw_cache = getCache('part2ch10_raw')

@raw_cache.memoize(typed=True)
def get_ct_raw_candidate(series_uid, center_xyz, width_irc):
  ct = getCt(series_uid)
  ct_chunk, center_irc = CT.get_raw_candidate(center_xyz, width_irc)
  return ct_chunk, center_irc



# TRAINING

In [ ]:
class LunaTrainingApp:

  def __init__(self, sys_argv=None):
    



4.3G	subset0/subset0.zip
